# Subsidy Efficiency Evaluation

**Definition**
```
Subsidy = (Reference price - End-user price) × Units consumed
```

---
### Libraries Import

In [ ]:
-- SELECT * 
-- FROM Fossils.fossil_fuel_subsidies 
-- -- limit 10
-- ;

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

### Database Import - SQL Query

In [ ]:
-- SELECT * 
-- FROM Fossils.fossil_fuel_subsidies_gdp
-- -- limit 10
-- ;

In [ ]:
-- SELECT * 
-- FROM Fossils.fossil_fuel_subsidies_per_capita
-- -- limit 10
-- ;

In [ ]:
SELECT DISTINCT 
    raw.Entity AS Entity, raw.Code AS Code, raw.Year AS Year,
    raw."12.c.1 - Fossil-fuel subsidies (consumption and production) (billions of nominal United States dollars) - ER_FFS_CMPT_CD" AS "Subsidies (Billions USD)", --ER_FFS_CMPT_CD, 
    gdp."12.c.1 - Fossil-fuel subsidies (consumption and production) as a proportion of total GDP (%) - ER_FFS_CMPT_GDP" AS "Subsidies (%GDP)", --ER_FFS_CMPT_GDP, 
    pc."12.c.1 - Fossil-fuel subsidies (consumption and production) per capita (nominal United States dollars) - ER_FFS_CMPT_PC_CD" AS "Subsidies per Capita (USD)" --ER_FFS_CMPT_PC_CD
FROM Fossils.fossil_fuel_subsidies AS raw
INNER JOIN Fossils.fossil_fuel_subsidies_gdp AS gdp
    ON raw.Entity = gdp.Entity 
    AND raw.Code = gdp.Code 
    AND raw.Year = gdp.Year
--     USING (Entity, Code, Year)    
INNER JOIN Fossils.fossil_fuel_subsidies_per_capita AS pc
    ON raw.Entity = pc.Entity 
    AND raw.Code = pc.Code 
    AND raw.Year = pc.Year
--     USING (Entity, Code, Year)    
;

- **Note**: The counts do not agree & match up between the tables, so some rows are lost due to incomplete matching during join (eg. missing data from certain years; may do loose/full join to find the lossy data)

### Dataloading from Excelsheets

In [ ]:
# Read from Excel
%ntbl pull datasets "Excel data/FossilFuelSubsidiesTracker_GlobalData.xlsx"
%ntbl pull datasets "Excel data/Fossil fuel consumption subsidies, 2010-2021.xlsx"

In [ ]:
subsidies_xlsx = pd.ExcelFile("../../datasets/Excel data/FossilFuelSubsidiesTracker_GlobalData.xlsx")
consumptionSubs_xlsx = pd.ExcelFile("../../datasets/Excel data/Fossil fuel consumption subsidies, 2010-2021.xlsx")

# May use either xlsxfile.parse(sheet_name=[...]) or pd.read_excel(sheet_name=[...]) onwards

In [ ]:
display(subsidies_xlsx.sheet_names, 
        consumptionSubs_xlsx.sheet_names)

In [ ]:
# df_subs_xlsx = pd.read_excel(subsidies_xlsx, sheet_name=None)

# NOTE: "Values in billion USD, nominal"
df_subs_glb = pd.read_excel(subsidies_xlsx, sheet_name='global_trends', skiprows=3)

# NOTE: units here are percentage values per-GDP (%GDP) and nominal USD per-capita ($USDpC)
df_subs_full = pd.read_excel(subsidies_xlsx, sheet_name='fulldata')

In [ ]:
# df_conSubs_xlsx = pd.read_excel(consumptionSubs_xlsx, sheet_name=None)

# NOTE: true header on (7+4)th row; rows[5, 9] represent Total (global) --> need to be extracted out
# NOTE: units here are in "Real 2021 million USD"
df_cs_subs = pd.read_excel(consumptionSubs_xlsx, sheet_name='Subsidies by country', skiprows=4)

# NOTE: low representation of countries listed (only 41 total); primarily/essentially consisted of 2nd- and 3rd-world countries
# Units here are listed/registered in row 4 (proper header)
df_cs_inds = pd.read_excel(consumptionSubs_xlsx, sheet_name='Indicators by country', skiprows=3)

# NOTE: units here are in "Real 2021 million USD"
df_cs_transoil = pd.read_excel(consumptionSubs_xlsx, sheet_name='Transport Oil Subsidies', skiprows=4)

---
### Preliminary EDA & Preprocessing

In [ ]:
# Formatting the year for easier analyses and later manipulation
df_join.Year = pd.to_datetime(df_join.Year.astype('int32'), format='%Y')
# df_join.Year = df_join.Year.astype('object')

In [ ]:
# display(df_raw.head())
# display(df_gdp.head())
# display(df_pc.head())
display(df_join.head())

- **Note**: The data contain regional, continental/international, and global (UN) aggregates -- proper analyses would require careful selection (inclusion or exclusion) of the particular rows

In [ ]:
# Basic summary of stats
df_join.info()
df_join.describe()

#### Simple Trends

In [ ]:
# Overall global annual trend
display(df_join[df_join.Entity == 'World'])

In [ ]:
# Trends from UN aggregates (continents & regions)
display(df_join[df_join.Code == ''])

In [ ]:
# Canada's annual trend
df_canada = df_join[df_join.Entity == 'Canada']
display(df_canada)

In [ ]:
# Filtering out main sets of interest
df_filt = df_join[(df_join.Entity == 'World') | 
                  (df_join.Code == 'CAN') | 
                  (df_join.Code == 'USA') | 
                  (df_join.Code == '')]
#                   (df_join.Entity == 'Northern America (UN)')]
# df_filt = df_join[[code in ['CAN', 'USA', 'OWID_WRL', ''] for code in df_join.Code.tolist()]]
display(df_filt)

#### Checking distributions

In [ ]:
# Basic distribution plots to look out for outliers & abnormalities
# The Year x-axis on the plot needs to be fixed as discrete time-points
sns.displot(df_join.Year.dt.year, discrete=True)

# sns.displot(df_join, x="ER_FFS_CMPT_CD")
# sns.displot(df_join, x="ER_FFS_CMPT_GDP")
# sns.displot(df_join, x="ER_FFS_CMPT_PC_CD")
sns.displot(df_join, x="Subsidies (Billions USD)")
sns.displot(df_join, x="Subsidies (%GDP)")
sns.displot(df_join, x="Subsidies per Capita (USD)")

- **Note**: The data may be incomplete for certain years, leading to the non-uniform distribution
- **Note**: Consider removing the zeroes (empty/non-reported values)

In [ ]:
# Same distributions on the filtered set
# The Year x-axis on the plot needs to be fixed as discrete time-points
sns.displot(df_filt.Year.dt.year, discrete=True)

# sns.displot(df_join, x="ER_FFS_CMPT_CD")
# sns.displot(df_join, x="ER_FFS_CMPT_GDP")
# sns.displot(df_join, x="ER_FFS_CMPT_PC_CD")
sns.displot(df_join, x="Subsidies (Billions USD)")
sns.displot(df_join, x="Subsidies (%GDP)")
sns.displot(df_join, x="Subsidies per Capita (USD)")

#### Aggregations

In [ ]:
df_gb_ent = df_join.groupby(['Entity', 'Code'])#.agg([sum, 'mean']).drop(['Year'], axis=1).reset_index()

In [ ]:
# display(df_gb_ent.sum())#.drop(['Year'], axis=1))
display(df_gb_ent.mean())#.drop(['Year'], axis=1))

In [ ]:
# Same aggregation but on the filtered set
display(df_filt[df_filt.Entity != 'World'].groupby(['Entity', 'Code']).mean())

In [ ]:
# Grouping by Year
# df_gb_yr = df_join.groupby(['Year'])#.agg(['sum', 'mean']).reset_index()
# Need to exclude UN aggregates for correct results
df_gb_yr = df_join[(df_join.Code != '') & (df_join.Entity != 'World')].groupby(['Year'])

In [ ]:
# display(df_gb_yr.sum())
display(df_gb_yr.mean())


### _On to the Excel files_

In [ ]:
# Sheet names for ease of reference
display(subsidies_xlsx.sheet_names, 
        consumptionSubs_xlsx.sheet_names)

File: _FossilFuelSubsidiesTracker_GlobalData_

In [ ]:
# NOTE: "Values in billion USD, nominal"
# df_subs_glb
# remove footer rows (idx 12 to 16/end; ie. select up to idx 12)
df_subs_glb = df_subs_glb.iloc[:12]
# Formatting the year for easier analyses and later manipulation
df_subs_glb.Year = pd.to_datetime(df_subs_glb.Year.astype('int32'), format='%Y')
display(df_subs_glb)

In [ ]:
# NOTE: units here are percentage values per-GDP (%GDP) and nominal USD per-capita ($USDpC)
# df_subs_full
# Formatting the year for easier analyses and later manipulation
df_subs_full.Year = pd.to_datetime(df_subs_full.Year.astype('int32'), format='%Y')
display(df_subs_full)

In [ ]:
display(df_subs_full.Country.nunique(), df_subs_full.Country.unique())

In [ ]:
df_subs_filt = df_subs_full[(df_subs_full.Country == "Canada") | 
                            (df_subs_full.Country == "United States")]
display(df_subs_filt)

To revisit: 
- ~~Graph out the trends & compare for global, Canada, US~~
- Relate back to the fossil_fuel_subsidies datasets

File: _Fossil fuel consumption subsidies, 2010-2021_

In [ ]:
# NOTE: true header on (7+4)th row; rows[5, 9] represent Total (global) --> need to be extracted out
# NOTE: units here are in "Real 2021 million USD"
df_cs_subs

In [ ]:
# NOTE: low representation of countries listed (only 41 total); primarily/essentially consisted of 2nd- and 3rd-world countries
# Units here are listed/registered in row 4 (proper header)
df_cs_inds

In [ ]:
# NOTE: units here are in "Real 2021 million USD"
df_cs_transoil

### TODOs & Future Considerations
- ~~Rankings & comparisons~~
    - ~~Canada's contributions/part in North America & World~~
- Sankey/Flow Diagrams to break down the details of subsidies/spendings
- Bubble plot (or some other insightful 3D plot) of UN regions/continents and World against North America (Canada & USA)
- Filter out zeroes and even out distributions